In [1]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import datetime

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier,GradientBoostingRegressor
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import accuracy_score


import tensorflow as tf
from tensorflow import keras
import seaborn as sbn

In [2]:
# import data TODO: Change the file path to the dataset this question needed
df = pd.read_csv('..\\source_data\\cf_gas_p2_df.csv')
df

,week,oil_futures,cpi,us_pop,public_roads,us_urban_pop,oil_production,djus_auto_index,auto_sales,us_real_gdp,usd_mex_exrate,gasoline
0,2009-01-04,40.83,93.262178,306035.0,4059352.000,244607104.0,4935,83.77,10.535732,14476.2498,13.657700,1.684
1,2009-01-11,36.51,91.817169,306085.5,4059506.692,244658433.9,4917,83.06,8.347124,16075.1555,13.899500,1.784
2,2009-01-18,46.47,91.122475,306136.0,4059661.385,244709763.8,5052,74.02,8.574011,15137.1916,14.025100,1.847
3,2009-01-25,41.68,87.888142,306186.5,4059816.077,244761093.7,5045,64.56,8.961935,15155.3440,14.255700,1.838
4,2009-02-01,40.17,82.715265,306237.0,4059970.769,244812423.6,5235,62.59,9.348876,15538.1713,14.119200,1.892
...,...,...,...,...,...,...,...,...,...,...,...,...
569,2019-12-01,59.20,112.084710,329314.0,4190039.827,270476205.2,12900,310.32,17.941803,19477.3091,19.286699,2.575
570,2019-12-08,60.07,109.471205,329335.8,4189928.462,270513569.8,12800,311.68,14.872529,19750.1111,18.997499,2.561
571,2019-12-15,60.44,108.183411,329357.6,4189817.096,270550934.3,12800,318.41,18.616565,20106.3904,18.938499,2.536
572,2019-12-22,61.72,108.707988,329379.4,4189705.731,270588298.9,12900,334.78,16.962709,20396.2701,18.844700,2.532


# Creating independent and dependent variables 

In [3]:
# calculate independent variables
periods = (1,2,3,5,10)
for i in periods:
    df['of_chg_{}'.format(i)] = df['oil_futures'].pct_change(periods = i)
    
# calculate dependent variable
df['gas_chg'] = df['gasoline'].pct_change(periods=1)

# eliminate the empty rows
df = df[11:]
df

,week,oil_futures,cpi,us_pop,public_roads,us_urban_pop,oil_production,djus_auto_index,auto_sales,us_real_gdp,usd_mex_exrate,gasoline,of_chg_1,of_chg_2,of_chg_3,of_chg_5,of_chg_10,gas_chg
11,2009-03-22,52.38,93.593485,306562.2,4061053.615,245171732.9,5432,65.02,10.622196,15121.9490,14.338800,1.962,0.025852,0.132541,0.150703,0.345146,0.434675,0.027225
12,2009-03-29,52.51,105.508484,306603.6,4061208.308,245223062.8,5480,71.65,7.626950,15312.9473,13.541800,2.046,0.002482,0.028398,0.135351,0.173146,0.129976,0.042813
13,2009-04-05,52.24,97.283677,306645.0,4061363.000,245274392.8,5469,79.32,9.304512,15389.4132,13.130000,2.037,-0.005142,-0.002673,0.023110,0.147627,0.253359,-0.004399
14,2009-04-12,50.33,79.216384,306699.5,4061517.692,245325722.7,5482,88.05,8.453785,14744.1587,13.097800,2.051,-0.036562,-0.041516,-0.039137,0.088216,0.252925,0.006873
15,2009-04-19,51.55,95.223458,306754.0,4061672.385,245377052.6,5421,89.79,9.155383,14624.3705,13.550500,2.059,0.024240,-0.013208,-0.018282,0.009597,0.374300,0.003901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,2019-12-01,59.20,112.084710,329314.0,4190039.827,270476205.2,12900,310.32,17.941803,19477.3091,19.286699,2.575,0.073047,0.024753,0.025641,0.053381,0.058845,-0.001551
570,2019-12-08,60.07,109.471205,329335.8,4189928.462,270513569.8,12800,311.68,14.872529,19750.1111,18.997499,2.561,0.014696,0.088816,0.039813,0.049441,0.137474,-0.005437
571,2019-12-15,60.44,108.183411,329357.6,4189817.096,270550934.3,12800,318.41,18.616565,20106.3904,18.938499,2.536,0.006159,0.020946,0.095523,0.047124,0.104936,-0.009762
572,2019-12-22,61.72,108.707988,329379.4,4189705.731,270588298.9,12900,334.78,16.962709,20396.2701,18.844700,2.532,0.021178,0.027468,0.042568,0.068375,0.147639,-0.001577


# Seperating training, validation, and test datasets

In [4]:
# create list of independent variable's name TODO: add the appropriate variable names to the list
#ind_var_name = ['of_chg_{}'.format(i) for i in periods]+['cpi','us_pop','public_roads','us_urban_pop','oil_production','djus_auto_index','auto_sales','us_real_gdp','usd_mex_exrate']
ind_var_name = ['of_chg_{}'.format(i) for i in periods]+['cpi','us_pop','public_roads',
                                                         'us_urban_pop','oil_production','auto_sales',
                                                         'us_real_gdp']
num_var =len(ind_var_name) # put number of variables here

# extract the values to X
X = df[['oil_futures'] + ind_var_name].values
X

array([[ 5.23800000e+01,  2.58519389e-02,  1.32540541e-01, ...,
         5.43200000e+03,  1.06221963e+01,  1.51219490e+04],
       [ 5.25100000e+01,  2.48186331e-03,  2.83979632e-02, ...,
         5.48000000e+03,  7.62694980e+00,  1.53129473e+04],
       [ 5.22400000e+01, -5.14187774e-03, -2.67277587e-03, ...,
         5.46900000e+03,  9.30451160e+00,  1.53894132e+04],
       ...,
       [ 6.04400000e+01,  6.15948061e-03,  2.09459459e-02, ...,
         1.28000000e+04,  1.86165647e+01,  2.01063904e+04],
       [ 6.17200000e+01,  2.11780278e-02,  2.74679541e-02, ...,
         1.29000000e+04,  1.69627090e+01,  2.03962701e+04],
       [ 6.30500000e+01,  2.15489307e-02,  4.31833223e-02, ...,
         1.29000000e+04,  1.78387504e+01,  1.88128854e+04]])

In [5]:
# create y for regressor, y_c for classifier
y = df['gas_chg']
y_c = (df['gas_chg'] > 0).values.astype('int')

# set training, validation, and test criterias
training_ratio = 0.7
validation_ratio = 0.15
test_ratio = 0.15

# check correctness of X and y
X.shape, y.shape, y_c.shape

((563, 13), (563,), (563,))

In [6]:
# create training, validation, and test sets for regressor
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_ratio, random_state = 42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = validation_ratio/(validation_ratio + training_ratio), random_state = 42)
print("X_train shape:\t", X_train.shape)
print("X_test shape:\t", X_test.shape)
print("X_val shape:\t", X_val.shape)
print("y_train shape:\t", y_train.shape)
print("y_val shape:\t", y_val.shape)
print("y_test shape:\t", y_test.shape)

X_train shape:	 (393, 13)
X_test shape:	 (85, 13)
X_val shape:	 (85, 13)
y_train shape:	 (393,)
y_val shape:	 (85,)
y_test shape:	 (85,)


In [7]:
# create training, validation, and test sets for classifier
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X, y_c, test_size = test_ratio, random_state = 42)
X_train_c, X_val_c, y_train_c, y_val_c = train_test_split(X_train_c, y_train_c, test_size = validation_ratio/(validation_ratio + training_ratio), random_state = 42)
print("X_train_c shape:", X_train_c.shape)
print("X_test_c shape:\t", X_test_c.shape)
print("X_val_c shape:\t", X_val_c.shape)
print("y_train_c shape:", y_train_c.shape)
print("y_val_c shape:\t", y_val_c.shape)
print("y_test_c shape:\t", y_test_c.shape)

X_train_c shape: (393, 13)
X_test_c shape:	 (85, 13)
X_val_c shape:	 (85, 13)
y_train_c shape: (393,)
y_val_c shape:	 (85,)
y_test_c shape:	 (85,)


# Principal Component Analysis (PCA)

In [8]:
# normalization for regressor set
for i in range(X_train.shape[1]):
    X_train_mean = X_train[:, i]. mean()
    X_train_std = X_train[:, i].std()
    X_train[:, i] = (X_train[:,i] - X_train_mean) / X_train_std
    X_test[:,i] = (X_test[:,i] - X_train_mean) / X_train_std
    X_val[:, i] = (X_val[:,i] - X_train_mean) / X_train_std

print("X_train shape:\t", X_train.shape)
print("X_val shape:\t", X_val.shape)

X_train shape:	 (393, 13)
X_val shape:	 (85, 13)


In [9]:
# normalization for classifier set
for i in range(X_train_c.shape[1]):
    X_train_c_mean = X_train_c[:, i]. mean()
    X_train_c_std = X_train_c[:, i].std()
    X_train_c[:, i] = (X_train_c[:,i] - X_train_c_mean) / X_train_c_std
    X_test_c[:,i] = (X_test_c[:,i] - X_train_c_mean) / X_train_c_std
    X_val_c[:, i] = (X_val_c[:,i] - X_train_c_mean) / X_train_c_std

print("X_train_c shape:", X_train_c.shape)
print("X_val_c shape:\t", X_val_c.shape)

X_train_c shape: (393, 13)
X_val_c shape:	 (85, 13)


In [10]:
# PCA for regressor
# fit on training set
scaler = StandardScaler()
scaler.fit(X_train)

# apply transform to training and test set
X_train_pca = scaler.transform(X_train)
X_test_pca = scaler.transform(X_test)
X_val_pca = scaler.transform(X_val)

# set PCA value
pca = PCA(0.95)

# fit PCA training set
pca.fit(X_train_pca)

# apply transform to training and test set
X_train_pca = pca.transform(X_train_pca)
X_test_pca = pca.transform(X_test_pca)
X_val_pca = pca.transform(X_val_pca)
print("X_train_pca shape:\t", X_train_pca.shape)
print("X_test_pca shape:\t", X_test_pca.shape)
print("X_val_pca shape:\t", X_val_pca.shape)
print("y_train shape:\t", y_train.shape)
print("y_val shape:\t", y_val.shape)
print("y_test shape:\t", y_test.shape)

X_train_pca shape:	 (393, 7)
X_test_pca shape:	 (85, 7)
X_val_pca shape:	 (85, 7)
y_train shape:	 (393,)
y_val shape:	 (85,)
y_test shape:	 (85,)


In [11]:
# PCA for classifier
# fit on training set_c
scaler = StandardScaler()
scaler.fit(X_train_c)

# apply transform to training and test set
X_train_c_pca = scaler.transform(X_train_c)
X_test_c_pca = scaler.transform(X_test_c)
X_val_c_pca = scaler.transform(X_val_c)

# set PCA value
pca = PCA(0.95)

# fit PCA training set
pca.fit(X_train_c_pca)

# apply transform to training and test set
X_train_c_pca = pca.transform(X_train_c_pca)
X_test_c_pca = pca.transform(X_test_c_pca)
X_val_c_pca = pca.transform(X_val_c_pca)
print("X_train_c_pca shape:\t", X_train_c_pca.shape)
print("X_test_c_pca shape:\t", X_test_c_pca.shape)
print("X_val_c_pca shape:\t", X_val_c_pca.shape)
print("y_train_c shape:", y_train_c.shape)
print("y_val_c shape:\t", y_val_c.shape)
print("y_test_c shape:\t", y_test_c.shape)

X_train_c_pca shape:	 (393, 7)
X_test_c_pca shape:	 (85, 7)
X_val_c_pca shape:	 (85, 7)
y_train_c shape: (393,)
y_val_c shape:	 (85,)
y_test_c shape:	 (85,)


In [12]:
num_var = X_train_c_pca.shape[1]
num_var

7

# Multiple linear regression

In [13]:
multiple_reg = LinearRegression().fit(X_train_pca, y_train)
score_multiple_reg = multiple_reg.score(X_test_pca, y_test)
print('score_multiple_reg:', "%.4f"%score_multiple_reg)

score_multiple_reg: 0.4322


# Multiple polynomial regression

In [14]:
#to prove this is the correct polynomial regression, set degree to 1
poly = PolynomialFeatures(1)
# polynomial transform
X_train_poly = poly.fit_transform(X_train_pca, y_train)
X_val_poly = poly.fit_transform(X_val_pca,y_val)
X_test_poly = poly.fit_transform(X_test_pca,y_test)

# fit to linear regression
polyreg = LinearRegression().fit(X_train_poly, y_train)

# predict using liear regression
score_poly_reg = polyreg.score(X_test_poly, y_test)
score_poly_reg

0.4321589599735277

In [15]:
# use val to find the appropriate degree
mpr = dict()
for i in range(2,6):
    degree=i
    poly = PolynomialFeatures(degree)
    # polynomial transform
    X_train_poly = poly.fit_transform(X_train_pca, y_train)
    X_val_poly = poly.fit_transform(X_val_pca,y_val)
    X_test_poly = poly.fit_transform(X_test_pca,y_test)

    # fit to linear regression
    polyreg = LinearRegression().fit(X_train_poly, y_train)

    # predict using liear regression
    score_poly_reg = polyreg.score(X_val_poly, y_val)
    print('score_poly_reg, degree',i,':', "%.4f"%score_poly_reg)
    mpr[f'{i}'] = score_poly_reg
    
max_degree = max(mpr, key = mpr.get)
print('max degree:', max_degree)

score_poly_reg, degree 2 : 0.3233
score_poly_reg, degree 3 : -1.2549
score_poly_reg, degree 4 : -62.6089
score_poly_reg, degree 5 : -2096.1610
max degree: 2


In [16]:
poly = PolynomialFeatures(int(max_degree))
# polynomial transform
X_train_poly = poly.fit_transform(X_train_pca, y_train)
X_val_poly = poly.fit_transform(X_val_pca,y_val)
X_test_poly = poly.fit_transform(X_test_pca,y_test)

# fit to linear regression
polyreg = LinearRegression().fit(X_train_poly, y_train)

# predict using liear regression
score_poly_reg = polyreg.score(X_test_poly, y_test)
score_poly_reg

0.4068729251920703

# Random forest classifier

In [17]:
# testing for different max depth
rfc = dict()
for i in range (1, 15):
    clf = RandomForestClassifier(max_depth=i, random_state=0).fit(X_train_c_pca, y_train_c)
    y_pred = clf.predict(X_val_c_pca)
    score_clf = accuracy_score(y_val_c, y_pred)
    rfc[f'{i}'] = score_clf
    #print('score_clf_',i,":", "%.4f"%score_clf)
max_depth = max(rfc, key=rfc.get)
print('max depth:', max_depth)

max depth: 1


In [18]:
# choose max_depth = 4
clf = RandomForestClassifier(max_depth=int(max_depth), random_state=0).fit(X_train_c_pca, y_train_c)
score_rf_clf = clf.score(X_test_c_pca, y_test_c)
print('score_rf_clf:', "%.4f"%score_rf_clf)

score_rf_clf: 0.6471


# Random forest regressor

In [19]:
clf_reg = RandomForestRegressor().fit(X_train_pca, y_train)
score_rf_reg = clf_reg.score(X_test_pca, y_test)
print('score_rf_reg:', "%.4f"%score_rf_reg)

score_rf_reg: 0.2639


# Logistic regression

In [20]:
logistic_model = LogisticRegression(penalty = 'l1', solver='liblinear', random_state=0).fit(X_train_c_pca, y_train_c)
score_logistic = logistic_model.score(X_test_c_pca, y_test_c)
print('score_logistic:', "%.4f"%score_logistic)

score_logistic: 0.7647


# Gradient boosting classifier

In [21]:
gra_clf = GradientBoostingClassifier(n_estimators = 200, learning_rate=0.50, max_depth=5, random_state=0).fit(X_train_c_pca, y_train_c)
score_gb_clf = gra_clf.score(X_test_c_pca, y_test_c)
score_gb_clf

0.7058823529411765

# Gradient boosting regressor 

In [22]:
gra_reg = GradientBoostingRegressor(random_state=0).fit(X_train_pca, y_train)
score_gb_reg = gra_reg.score(X_test_pca, y_test)
score_gb_reg

0.3547136867120756

# XGBoost classifier

In [23]:
xgb_clf = XGBClassifier().fit(X_train_c_pca, y_train_c)
score_xgb_clf = xgb_clf.score(X_test_c_pca, y_test_c)
score_xgb_clf

[17:04:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\lemon\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.7058823529411765

# XGBoost regressor

In [24]:
xgb_reg = XGBRegressor(random_state = 0).fit(X_train_pca, y_train)
score_xgb_reg = xgb_reg.score(X_test_pca, y_test)
score_xgb_reg

0.22434733981830735

# AdaBoost classifier

In [25]:
adb_clf = AdaBoostClassifier().fit(X_train_c_pca, y_train_c)
score_adb_clf = adb_clf.score(X_test_c_pca, y_test_c)
score_adb_clf

0.6588235294117647

# AdaBoost regressor

In [26]:
adb_reg = AdaBoostRegressor().fit(X_train_pca, y_train)
score_adb_reg = adb_reg.score(X_test_pca, y_test)
score_adb_reg

0.13013166360389572

# Support vector classification

In [27]:
# Try different kernels
kernels = ('linear', 'poly', 'rbf')

for k in kernels:
    svm_clf = SVC(kernel=k).fit(X_train_c_pca, y_train_c)
    y_pred = svm_clf.predict(X_test_c_pca)
    vars()[f'score_svc_{k}']= accuracy_score(y_test_c, y_pred)
    
print(score_svc_linear)
print(score_svc_poly)
print(score_svc_rbf)

0.7411764705882353
0.7058823529411765
0.7764705882352941


# Gaussian Naive Bayes

In [28]:
gnb_clf = GaussianNB().fit(X_train_c_pca, y_train_c)
score_gnb = gnb_clf.score(X_test_c_pca, y_test_c)
print(score_gnb)

0.7294117647058823


# Neuro networks

In [29]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(10, input_dim=num_var, activation='relu'))
model.add(keras.layers.Dense(5, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train_c_pca, y_train_c, epochs=50, validation_data=(X_val_c_pca, y_val_c))

Epoch 1/50
13/13 [==============================] - 2s 92ms/step - loss: 0.8342 - accuracy: 0.4217 - val_loss: 0.8412 - val_accuracy: 0.4118
Epoch 2/50
13/13 [==============================] - 0s 4ms/step - loss: 0.8374 - accuracy: 0.3772 - val_loss: 0.8161 - val_accuracy: 0.3882
Epoch 3/50
13/13 [==============================] - 0s 4ms/step - loss: 0.8089 - accuracy: 0.3970 - val_loss: 0.7972 - val_accuracy: 0.3529
Epoch 4/50
13/13 [==============================] - 0s 3ms/step - loss: 0.7536 - accuracy: 0.4642 - val_loss: 0.7820 - val_accuracy: 0.3647
Epoch 5/50
13/13 [==============================] - 0s 4ms/step - loss: 0.7270 - accuracy: 0.4703 - val_loss: 0.7686 - val_accuracy: 0.3882
Epoch 6/50
13/13 [==============================] - 0s 5ms/step - loss: 0.7432 - accuracy: 0.4264 - val_loss: 0.7568 - val_accuracy: 0.3765
Epoch 7/50
13/13 [==============================] - 0s 4ms/step - loss: 0.7312 - accuracy: 0.4325 - val_loss: 0.7465 - val_accuracy: 0.3765
Epoch 8/50
13/13 [=

In [30]:
loss, score_neuro = model.evaluate(X_test_c_pca, y_test_c)
print(score_neuro)

3/3 [==============================] - 0s 998us/step - loss: 0.5366 - accuracy: 0.7882
0.7882353067398071


# Summary

In [31]:
reg = {
    'Multiple linear regression': score_multiple_reg,
    'Multiple polynomial regression': score_poly_reg,
    'Random forest regressor': score_rf_reg,
    'Gradient boosting regressor': score_gb_reg,
    'XGBoost regressor': score_xgb_reg,
    'AdaBoost regressor': score_adb_reg
}
reg_df = pd.DataFrame.from_dict(reg, orient='index', columns = ['Score'])
reg_df.index.name = 'Regression models'
#reg_df.sort_values(by=['Score'], ascending=False)
reg_df

,Score
Regression models,
Multiple linear regression,0.432159
Multiple polynomial regression,0.406873
Random forest regressor,0.263905
Gradient boosting regressor,0.354714
XGBoost regressor,0.224347
AdaBoost regressor,0.130132


In [32]:
clf = {
    'Logistic regression': score_logistic,
    'Random forest classifier': score_rf_clf,
    'Gradient boosting classifier': score_gb_clf,
    'XGBoost classifier': score_xgb_clf,
    'AdaBoost classifier': score_adb_clf,
    'Gaussian naive Bayes': score_gnb, 
    'SVC linear': score_svc_linear,
    'SVC polynomial': score_svc_poly, 
    'SVC RBF': score_svc_rbf,
    'Neuro network' : score_neuro
}
clf_df = pd.DataFrame.from_dict(clf, orient='index',columns = ['Score'])
clf_df.index.name = 'Classifier models'
#clf_df.sort_values(by=['Score'], ascending=False)
clf_df

,Score
Classifier models,
Logistic regression,0.764706
Random forest classifier,0.647059
Gradient boosting classifier,0.705882
XGBoost classifier,0.705882
AdaBoost classifier,0.658824
Gaussian naive Bayes,0.729412
SVC linear,0.741176
SVC polynomial,0.705882
SVC RBF,0.776471
